# NB1.5 — Streaming vs Batch (WebSockets & Polling)

En esta práctica implementamos **Streaming** con WebSockets y **Batch frecuente** con HTTP Polling. Guardaremos todo en **JSON Lines** en `../../data/` y compararemos latencia y continuidad.

**Objetivos**
- Entender diferencias clave **Batch vs Streaming**.
- Ejecutar un cliente **WebSocket** para ingesta continua.
- Ejecutar **HTTP Polling** como micro-batch.
- Comparar resultados en pandas.


## 1) Dependencias
Instalamos librerías requeridas para los scripts en `src/streaming/`.

In [1]:
!pip install -q websockets requests pandas

## 2) Streaming (WebSocket)

**Salida:** `../../data/stream_ws_YYYY-MM-DD.jsonl`

In [3]:
# Limita duración/eventos para la clase
!WS_MAX_EVENTS=150 WS_MAX_SECONDS=120 python ../../src/streaming/stream_dual_ws.py

/Users/eduardo/Developer/TSW/fundamentos/fundamentos-ingenieria-datos/notebooks/tutoriales/../../src/streaming/stream_dual_ws.py:8: DeprecationWarning: websockets.exceptions.InvalidStatusCode is deprecated
  from websockets.exceptions import ConnectionClosed, InvalidStatusCode
[DONE] Eventos: 150
/Users/eduardo/Developer/TSW/fundamentos/fundamentos-ingenieria-datos/data/stream_ws_2025-09-29.jsonl


## 3) Polling (HTTP)

**Salida:** `../../data/poll_bitcoin_YYYY-MM-DD.jsonl`

In [4]:
!python ../../src/streaming/poll_coincap_http.py

[POLL] 1/20 -> {'ts': '2025-09-30T04:33:18.039576+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114147.47}
[POLL] 2/20 -> {'ts': '2025-09-30T04:33:23.723831+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114147.47}
[POLL] 3/20 -> {'ts': '2025-09-30T04:33:29.100095+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114147.47}
[POLL] 4/20 -> {'ts': '2025-09-30T04:33:34.453986+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114147.48}
[POLL] 5/20 -> {'ts': '2025-09-30T04:33:39.819876+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114170.45}
[POLL] 6/20 -> {'ts': '2025-09-30T04:33:45.173163+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114185.78}
[POLL] 7/20 -> {'ts': '2025-09-30T04:33:50.525958+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 'price_usd': 114207.99}
[POLL] 8/20 -> {'ts': '2025-09-30T04:33:55.891138+00:00', 'source': 'binance', 'instrument': 'BTCUSDT', 

## 4) Cargar y comparar en pandas
Leemos ambos `.jsonl` y observamos diferencias de granularidad/latencia (aprox.).

In [5]:
from pathlib import Path
import pandas as pd

data_dir = Path("../../data")

# STREAM
stream_file = sorted(data_dir.glob("stream_ws_*.jsonl"))[-1]
df_stream = pd.read_json(stream_file, lines=True)
print("STREAM:", stream_file)
display(df_stream.head())

# POLL 
poll_files = sorted(list(data_dir.glob("poll_binance_*.jsonl")) + list(data_dir.glob("poll_coincap_*.jsonl")))
poll_file = poll_files[-1]
df_poll = pd.read_json(poll_file, lines=True)
print("POLL:", poll_file)
display(df_poll.head())


STREAM: ../../data/stream_ws_2025-09-29.jsonl


,_probe,ts,source,instrument,price,currency,qty,trade_id
0,1.0,2025-09-30T04:29:24.413211+00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2025-09-30T04:29:25.680613+00:00,binance,BTCUSDT,114200.00,USDT,0.00638,5.267491e+09
2,NaN,2025-09-30T04:29:26.098653+00:00,binance,BTCUSDT,114200.01,USDT,0.00066,5.267491e+09
3,NaN,2025-09-30T04:29:26.398568+00:00,binance,BTCUSDT,114200.01,USDT,0.00178,5.267491e+09
4,NaN,2025-09-30T04:29:26.472646+00:00,binance,BTCUSDT,114200.00,USDT,0.00017,5.267491e+09


POLL: ../../data/poll_binance_BTCUSDT_2025-09-29.jsonl


,ts,source,instrument,price_usd
0,2025-09-30T04:33:18.039576+00:00,binance,BTCUSDT,114147.47
1,2025-09-30T04:33:23.723831+00:00,binance,BTCUSDT,114147.47
2,2025-09-30T04:33:29.100095+00:00,binance,BTCUSDT,114147.47
3,2025-09-30T04:33:34.453986+00:00,binance,BTCUSDT,114147.48
4,2025-09-30T04:33:39.819876+00:00,binance,BTCUSDT,114170.45


## 5) Reflexión (respuestas cortas)
**¿Quién tiene menor latencia?**
El websocket tiene menor latencia ya que la conexión es persistente o en tiempo real  

**¿Qué pasa si el stream se cae?** 
Realiza reintentos de conexion, donde cada reintento aumenta exponencialmente el tiempo de espera

**¿Cuál genera más duplicados o huecos temporales?**
Duplicados: WS, huecos temporales: Http polling
